In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [33]:
post_df=pd.read_csv("../data/processed/posts.csv")
user_df=pd.read_csv("../data/processed/users.csv")
post_df.dropna(subset=['CreationDate'],inplace=True)

post_df.drop(columns=['LastEditorUserId','LastEditDate','LastActivityDate','Id'],inplace=True)

post_df.dropna(subset=['CreationDate','OwnerUserId'],inplace=True)

post_df[['OwnerUserId']]=post_df[['OwnerUserId']].astype(int)

post_df=post_df[post_df['OwnerUserId']!=-1].reset_index(drop=True)
user_df=user_df[user_df['Id']!=-1].reset_index(drop=True)
#print(user_df)

In [34]:
post_oid_cd_df=post_df[['OwnerUserId','CreationDate']].copy()
user_oid_cd_df=user_df[['Id','CreationDate']].copy()
user_oid_cd_df.rename(mapper={'Id':'OwnerUserId'},axis=1,inplace=True)
final_df=post_oid_cd_df.append(user_oid_cd_df, ignore_index=True)


In [35]:
final_df.dropna(how='any',inplace=True)
final_df=final_df.groupby('OwnerUserId').filter(lambda x:x['CreationDate'].count()>=2)
final_df['CreationDate']=pd.to_datetime(final_df['CreationDate'])
final_df['diff'] = final_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True]).groupby('OwnerUserId')['CreationDate'].diff()
final_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True],inplace=True)


In [36]:
df=final_df.copy()
df.reset_index(drop=True,inplace=True)
df.dropna(how='any',inplace=True)

In [37]:
k=3
lik=[i for i in range(k)]
k_df=df.groupby('OwnerUserId').filter(lambda x:x['CreationDate'].count()>=k)
k_df=k_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True]).groupby('OwnerUserId').nth(lik).reset_index()
k_df['diff']=k_df['diff']/np.timedelta64(1, 'm')


In [38]:
fin_df=k_df[['OwnerUserId','diff']].copy().reset_index(drop=True)

In [39]:
x=fin_df.sort_values(["OwnerUserId","diff"]).groupby('OwnerUserId')['diff'].apply(lambda df: df.reset_index(drop=True)).unstack()
print(x.head(10))
print(x.shape)

                        0              1             2
OwnerUserId                                           
11           39382.068050  166104.547617  1.940929e+06
14              41.963717     379.793667  9.769404e+02
21              11.426283     781.536717  1.486153e+03
26             108.443933     501.561450  5.160215e+02
34            3873.325950  347053.706500  1.581795e+06
36              16.318067    6910.978383  1.697913e+05
53               0.000000     442.966950  1.163099e+03
59               3.133833     626.378383  6.411663e+03
62             128.849717     701.314833  4.265212e+04
75               0.000000      50.242383  5.416974e+05
(1712, 3)
